# **OBJECT TRACKING**

Object Tracking Techniques:
* Optical Flow
* Meanshift and Camshift
* Using built-in APIs

### **OPTICAL FLOW:**

Optical flow is the motion of objects between the consecutive frames of the sequence, caused by the relative motion between the camera and the object. It can be of two types-Sparse Optical flow and Dense Optical flow.

Read more at: https://learnopencv.com/optical-flow-in-opencv/

Optical Flow analysis has a few assumptions: 
* The pixel intensities of an object do not change between consecutive frames.
* Neighbouring pixels have similar motions.

* The optical flow methods in OpenCV will first take in a given set of points and a frame.
* Then it will attempt to find those points in the next frame.
* It is up to the user to supply the points to track.

We will make use of *Lucas-Kanade* method which computes optical flow for a **sparse** feature set. 
> Meaning only the points it was told to track.

* We can use Gunner Farneback's algorithm (also built in to OpenCV) to calculate dense optical flow.
* This dense optical flow will calculate flow for all points in an image. It will color them black if no flow (no movement) is detected.


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [4]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Points to track:
prevPoints = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:

    ret, frame = cap.read() #current frame

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate Optical flow:
    nextPoints, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPoints, None, **lk_params)

    good_new = nextPoints[status==1]
    good_prev = prevPoints[status==1]

    for i, (new,prev) in enumerate(zip(good_new, good_prev)):

        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        mask = cv2.line(mask, (x_new,y_new), (x_prev, y_prev), (0,255,0), 3)

        frame = cv2.circle(frame, (x_new,y_new), 8, (0,0,255), -1)

    img = cv2.add(frame, mask)
    cv2.imshow('Tracking', img)

    k =cv2.waitKey(30)  & 0xFF

    if k == 27:
        break

    prev_gray = frame.copy()
    prevPoints = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\video\src\lkpyramid.cpp:1379: error: (-215:Assertion failed) prevPyr[level * lvlStep1].type() == nextPyr[level * lvlStep2].type() in function 'cv::`anonymous-namespace'::SparsePyrLKOpticalFlowImpl::calc'


: 